In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import os
import warnings
import time
import matplotlib.pyplot as plt
import concurrent.futures
import pickle

In [3]:
warnings.filterwarnings('ignore', category=FutureWarning)

# Step 1: Load and Profile Raw Data
file_path = r'C:\Users\DELL\Downloads\raw_nse_insider.csv'

try:
    insider_df = pd.read_csv(file_path, low_memory=False)
    print("Successfully loaded the dataset.")
    print(f"The dataset has {insider_df.shape[0]} rows and {insider_df.shape[1]} columns.")
    print("First 5 rows of the data:")
    print(insider_df.head())
    print("\nDataset Information:")
    insider_df.info()

    df_raw = pd.read_csv(file_path, low_memory=False)
    df_raw.columns = [col.strip() for col in df_raw.columns]
    print("--- DATASET PROFILE ---")
    print(f"Total Rows: {len(df_raw)}")
    print(f"Total Columns: {len(df_raw.columns)}")
    print("\n--- Column-wise Unique Values ---")
    for col in df_raw.columns:
        if df_raw[col].nunique() < 50:
            print(f"\nColumn: '{col}'")
            print("---------------------------------")
            unique_values = df_raw[col].unique().tolist()
            unique_values = ['<NaN>' if pd.isna(x) else x for x in unique_values]
            print(unique_values)
        else:
            print(f"\nColumn: '{col}' (has >50 unique values, showing a sample)")
            print("---------------------------------")
            sample_values = df_raw[col].dropna().sample(5).tolist()
            print(f"Sample values: {sample_values}")

except FileNotFoundError:
    print(f"FATAL ERROR: The file '{file_path}' was not found. Please check the name and location.")
    insider_df = pd.DataFrame()
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded the dataset.
The dataset has 51881 rows and 18 columns.
First 5 rows of the data:
  SYMBOL \n                                      COMPANY \n  \
0       M&M                     Mahindra & Mahindra Limited   
1       M&M                     Mahindra & Mahindra Limited   
2       M&M                     Mahindra & Mahindra Limited   
3    M&MFIN  Mahindra & Mahindra Financial Services Limited   
4       IKS          Inventurus Knowledge Solutions Limited   

                    NAME OF THE ACQUIRER/DISPOSER \n  \
0  Mahindra & Mahindra Employees' Stock Option Trust   
1                                         Alok Kumar   
2                                       Ninad Butala   
3  Mahindra & Mahindra Financial Services Limited...   
4                                    Dinesh Rijhwani   

            CATEGORY OF PERSON \n TYPE OF SECURITY (PRIOR) \n  \
0                           Other               Equity Shares   
1  Employees/Designated Employees               Equi

In [4]:
# Step 2: Clean the Data 
if not insider_df.empty:
    print(f"--- SCRIPT START ---")
    print(f"Successfully loaded '{file_path}'. Initial dataset has {len(insider_df)} rows.")
    insider_df.columns = [col.strip() for col in insider_df.columns]
    column_map = {
        'SYMBOL': 'symbol', 'COMPANY': 'company_name', 'NAME OF THE ACQUIRER/DISPOSER': 'insider_name',
        'CATEGORY OF PERSON': 'insider_category', 'NO. OF SECURITIES (ACQUIRED/DISPLOSED)': 'quantity',
        'VALUE OF SECURITY (ACQUIRED/DISPLOSED)': 'value', 'ACQUISITION/DISPOSAL TRANSACTION TYPE': 'transaction_type',
        '% POST': 'post_holding_pct', 'DATE OF ALLOTMENT/ACQUISITION FROM': 'transaction_date',
        'DATE OF INTIMATION TO COMPANY': 'broadcast_date', 'MODE OF ACQUISITION': 'mode'
    }
    insider_df.rename(columns=column_map, inplace=True)
    final_columns = [
        'symbol', 'company_name', 'insider_name', 'insider_category', 'quantity', 'value', 
        'transaction_type', 'post_holding_pct', 'transaction_date', 'broadcast_date', 'mode'
    ]
    existing_columns = [col for col in final_columns if col in insider_df.columns]
    insider_df = insider_df[existing_columns].copy()

    insider_df.replace(['-', 'Nil'], np.nan, inplace=True)
    insider_df['transaction_date'] = pd.to_datetime(insider_df['transaction_date'], errors='coerce', dayfirst=True)
    insider_df['broadcast_date'] = pd.to_datetime(insider_df['broadcast_date'], errors='coerce', dayfirst=True)
    for col in ['value', 'quantity', 'post_holding_pct']:
        insider_df[col] = pd.to_numeric(insider_df[col].astype(str).str.replace(',', '', regex=False), errors='coerce')
    print(f"Rows after initial cleaning & type conversion: {len(insider_df)}")

    insider_df.dropna(subset=['symbol', 'transaction_date', 'broadcast_date', 'quantity', 'value', 'transaction_type'], inplace=True)
    print(f"Rows after dropping critical NaNs: {len(insider_df)}")

    insider_df = insider_df[(insider_df['quantity'] > 0) & (insider_df['value'] > 0)]
    insider_df['price'] = insider_df['value'] / insider_df['quantity']
    insider_df = insider_df[(insider_df['price'] > 0) & np.isfinite(insider_df['price'])]
    print(f"Rows after filtering for valid trades: {len(insider_df)}")

    insider_df = insider_df[insider_df['transaction_type'].isin(['Buy', 'Sell'])]
    print(f"Rows after keeping only 'Buy'/'Sell' transactions: {len(insider_df)}")

    high_signal_categories = ['Promoters', 'Director', 'Promoter Group', 'Employees']  # Relaxed: Added 'Employees'
    insider_df = insider_df[insider_df['insider_category'].isin(high_signal_categories)]
    print(f"Rows after filtering for high-signal categories: {len(insider_df)}")

    market_transactions = ['Market Purchase', 'Market Sale']
    insider_df = insider_df[insider_df['mode'].isin(market_transactions)]
    print(f"Rows after filtering for Market Purchases/Sales: {len(insider_df)}")

    # Relaxed insider filter: >=1 unique insiders
    insider_df = insider_df.groupby(['symbol', 'broadcast_date']).filter(lambda x: len(x['insider_name'].unique()) >= 1).reset_index(drop=True)
    print(f"Rows after relaxed insider filtering: {len(insider_df)}")

    insider_df.sort_values(by='broadcast_date', inplace=True)
    insider_df.drop_duplicates(inplace=True)
    insider_df.reset_index(drop=True, inplace=True)
    print(f"\n--- Data Cleaning Complete ---")
    print(f"Final cleaned dataset has {len(insider_df)} rows.")

    cleaned_file_path = 'cleaned_insider_data.csv'
    insider_df.to_csv(cleaned_file_path, index=False)
    print(f"Cleaned data saved to '{cleaned_file_path}'.")

    if not insider_df.empty:
        print("\nFinal Cleaned Data Head:")
        print(insider_df.head())
        print("\nFinal Cleaned Data Info:")
        insider_df.info()
    else:
        print("\nWarning: The final DataFrame is empty. Check the row counts at each step.")

--- SCRIPT START ---
Successfully loaded 'C:\Users\DELL\Downloads\raw_nse_insider.csv'. Initial dataset has 51881 rows.


C:\Users\DELL\AppData\Local\Temp\ipykernel_67672\3174562842.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  insider_df['transaction_date'] = pd.to_datetime(insider_df['transaction_date'], errors='coerce', dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_67672\3174562842.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  insider_df['broadcast_date'] = pd.to_datetime(insider_df['broadcast_date'], errors='coerce', dayfirst=True)


Rows after initial cleaning & type conversion: 51881
Rows after dropping critical NaNs: 51667
Rows after filtering for valid trades: 49490
Rows after keeping only 'Buy'/'Sell' transactions: 45327
Rows after filtering for high-signal categories: 16332
Rows after filtering for Market Purchases/Sales: 12944
Rows after relaxed insider filtering: 12944

--- Data Cleaning Complete ---
Final cleaned dataset has 12728 rows.
Cleaned data saved to 'cleaned_insider_data.csv'.

Final Cleaned Data Head:
       symbol                        company_name  \
0    CONFIPET  Confidence Petroleum India Limited   
1     ALMONDZ   Almondz Global Securities Limited   
2    ARMANFIN    Arman Financial Services Limited   
3  MAHSEAMLES        Maharashtra Seamless Limited   
4   VINEETLAB         Vineet Laboratories Limited   

                                 insider_name insider_category  quantity  \
0         Essenn LPG Bottling Private Limited   Promoter Group  486359.0   
1  Avonmore Capital & Management 

In [5]:
# Step 3: Enrich Data with Relaxed Filters
try:
    insider_df = pd.read_csv('cleaned_insider_data.csv', parse_dates=['transaction_date', 'broadcast_date'])
    print(f"Successfully loaded 'cleaned_insider_data.csv' with {len(insider_df)} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'cleaned_insider_data.csv' not found. Please run the cleaning script first.")
    insider_df = pd.DataFrame()

print("Downloading Nifty 50 data from yfinance...")
try:
    start_date = insider_df['broadcast_date'].min() - pd.Timedelta(days=90)
    end_date = insider_df['broadcast_date'].max()
    nifty_df = yf.download('^NSEI', start=start_date, end=end_date, auto_adjust=False)
    nifty_df.index.name = 'Date'
    nifty_df['nifty_returns'] = nifty_df['Adj Close'].pct_change()
    print("Successfully downloaded Nifty 50 benchmark data.")
except Exception as e:
    print(f"FATAL ERROR: Could not download Nifty 50 data. Error: {e}")
    nifty_df = pd.DataFrame()

def sanity_check_price_data(df, ticker_name):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    required_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    if not all(col in df.columns for col in required_cols):
        print(f"  - Sanity check failed: Missing required columns for {ticker_name}.")
        return None
    df.ffill(inplace=True)
    price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
    df[price_cols] = df[price_cols].replace(0, np.nan).ffill()
    df.dropna(subset=price_cols, inplace=True)
    return df

def calculate_market_metrics(stock_df, nifty_df, ticker):
    stock_df['returns'] = stock_df['Adj Close'].pct_change()
    high_low = stock_df['High'] - stock_df['Low']
    high_prev_close = np.abs(stock_df['High'] - stock_df['Adj Close'].shift())
    low_prev_close = np.abs(stock_df['Low'] - stock_df['Adj Close'].shift())
    tr = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1)
    stock_df['atr_10'] = tr.rolling(window=10).mean()
    stock_df['daily_value'] = stock_df['Adj Close'] * stock_df['Volume']
    stock_df['adv_10'] = stock_df['daily_value'].rolling(window=10).mean()
    stock_df['volatility_60'] = stock_df['returns'].rolling(window=60).std() * np.sqrt(252)  # Changed to 60 for consistency
    combined_returns = pd.concat([stock_df['returns'], nifty_df['nifty_returns']], axis=1).dropna()
    rolling_cov = combined_returns['returns'].rolling(window=60).cov(combined_returns['nifty_returns'])
    rolling_var = combined_returns['nifty_returns'].rolling(window=60).var()
    stock_df['beta_60'] = rolling_cov / rolling_var
    
    try:
        market_cap = yf.Ticker(ticker).info.get('marketCap', 0) / 10000000  # In Cr
        stock_df['market_cap_cr'] = market_cap
    except:
        stock_df['market_cap_cr'] = 0
    
    return stock_df[['atr_10', 'adv_10', 'volatility_60', 'beta_60', 'market_cap_cr']]

if not insider_df.empty and not nifty_df.empty:
    print("\nStarting data enrichment process...")
    unique_symbols = insider_df['symbol'].unique()
    # For testing: Reduce to a sample for faster iteration
    # unique_symbols = unique_symbols[:50]  # Uncomment to sample 50 symbols

    # Caching: Check for cached price data
    cache_file = 'price_data_cache.pkl'
    if os.path.exists(cache_file):
        print("Loading cached price data...")
        with open(cache_file, 'rb') as f:
            all_price_data = pickle.load(f)
    else:
        print("Downloading batched price data for all symbols...")
        tickers = [f"{s}.NS" for s in unique_symbols] + [f"{s}.BO" for s in unique_symbols]  # Include both suffixes
        all_price_data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False, progress=True, group_by='ticker')
        with open(cache_file, 'wb') as f:
            pickle.dump(all_price_data, f)
        print("Cached price data saved.")

    def process_symbol(symbol):
        try:
            # Try .NS first, then .BO
            ticker_ns = f'{symbol}.NS'
            if ticker_ns in all_price_data:
                stock_df_raw = all_price_data[ticker_ns]
            else:
                ticker_bo = f'{symbol}.BO'
                if ticker_bo in all_price_data:
                    stock_df_raw = all_price_data[ticker_bo]
                    ticker_ns = ticker_bo
                else:
                    return None

            if stock_df_raw.empty:
                return None

            stock_df = sanity_check_price_data(stock_df_raw.copy(), symbol)
            if stock_df is None or stock_df.empty:
                return None

            metrics_df = calculate_market_metrics(stock_df, nifty_df, ticker_ns)
            
            trades_for_symbol = insider_df[insider_df['symbol'] == symbol].copy()
            enriched_trades = pd.merge_asof(
                trades_for_symbol.sort_values('broadcast_date'),
                metrics_df.sort_index(),
                left_on='broadcast_date',
                right_index=True,
                direction='backward'
            )
            return enriched_trades
        except Exception as e:
            print(f"  - An unexpected error occurred for {symbol}: {e}")
            return None

    # Parallel processing
    all_enriched_trades = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_symbol, unique_symbols))
        all_enriched_trades = [res for res in results if res is not None]

    if all_enriched_trades:
        enriched_df = pd.concat(all_enriched_trades, ignore_index=True)
        enriched_df.dropna(subset=['atr_10', 'adv_10', 'volatility_60', 'beta_60', 'market_cap_cr'], inplace=True)
        final_file_path = 'backtest_ready_data.csv'
        enriched_df.to_csv(final_file_path, index=False)
        print("\n--- Data Enrichment Complete ---")
        print(f"Final dataset ready for back-testing has {len(enriched_df)} rows.")
        print(f"Saved to '{final_file_path}'.")
        print("\nFinal Enriched Data Head:")
        print(enriched_df.head())
    else:
        print("\nCould not enrich any data. Please check for errors.")

Successfully loaded 'cleaned_insider_data.csv' with 12728 rows.



*********************100%***********************]  1 of 1 completed

Successfully downloaded Nifty 50 benchmark data.

Starting data enrichment process...
Loading cached price data...

--- Data Enrichment Complete ---
Final dataset ready for back-testing has 12297 rows.
Saved to 'backtest_ready_data.csv'.

Final Enriched Data Head:
     symbol                        company_name  \
1  CONFIPET  Confidence Petroleum India Limited   
2  CONFIPET  Confidence Petroleum India Limited   
3  CONFIPET  Confidence Petroleum India Limited   
4  CONFIPET  Confidence Petroleum India Limited   
5  CONFIPET  Confidence Petroleum India Limited   

                              insider_name insider_category  quantity  \
1      ESSENN LPG BOTTLING PRIVATE LIMITED   Promoter Group  153000.0   
2  CONFIDENCE LPG BOTTLING PRIVATE LIMITED   Promoter Group  605584.0   
3  CONFIDENCE LPG BOTTLING PRIVATE LIMITED   Promoter Group   26000.0   
4  CONFIDENCE LPG BOTTLING PRIVATE LIMITED   Promoter Group  100000.0   
5  CONFIDENCE LPG BOTTLING PRIVATE LIMITED   Promoter Group  52

In [6]:
# Step 4: Load Enriched Data and Display Info
df = pd.read_csv('backtest_ready_data.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12297 entries, 0 to 12296
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   symbol            12297 non-null  object 
 1   company_name      12297 non-null  object 
 2   insider_name      12294 non-null  object 
 3   insider_category  12297 non-null  object 
 4   quantity          12297 non-null  float64
 5   value             12297 non-null  float64
 6   transaction_type  12297 non-null  object 
 7   post_holding_pct  12294 non-null  float64
 8   transaction_date  12297 non-null  object 
 9   broadcast_date    12297 non-null  object 
 10  mode              12297 non-null  object 
 11  price             12297 non-null  float64
 12  atr_10            12297 non-null  float64
 13  adv_10            12297 non-null  float64
 14  volatility_60     12297 non-null  float64
 15  beta_60           12297 non-null  float64
 16  market_cap_cr     12297 non-null  float6

In [7]:
# Step 5: Backtest with Adjustments 
start_time = time.time()

try:
    data_df = pd.read_csv('backtest_ready_data.csv', parse_dates=['broadcast_date'])
    data_df['broadcast_date'] = data_df['broadcast_date'].dt.normalize()
    print(f"Loaded 'backtest_ready_data.csv' with {len(data_df)} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'backtest_ready_data.csv' not found. Exiting.")
    data_df = pd.DataFrame()

if not data_df.empty:
    initial_capital = 10000000
    max_active_positions = 20  
    transaction_cost_pct = 0.001  
    historical_events_lookback = 1  
    momentum_decay_min_hold_days = 20  
    risk_free_rate = 0.05  

    cash = initial_capital
    holdings = {}
    trade_log = []

    print("\nFetching all required price data...")
    sim_start_date = data_df['broadcast_date'].min() - pd.Timedelta(days=180)
    sim_end_date = data_df['broadcast_date'].max() + pd.Timedelta(days=60)
    unique_symbols = data_df['symbol'].unique()
    tickers_ns = [f"{s}.NS" for s in unique_symbols]
    all_price_data = yf.download(tickers_ns, start=sim_start_date, end=sim_end_date, auto_adjust=False, progress=True, group_by='ticker')

    price_data_cache = {}
    for symbol in unique_symbols:
        try:
            symbol_data = all_price_data[f"{symbol}.NS"]
            if not symbol_data.dropna().empty:
                price_data_cache[symbol] = symbol_data
        except KeyError:
            continue
    print(f"Successfully fetched and cached price data for {len(price_data_cache)} symbols.")

    print("Finding all potential historical signals...")
    all_signals = []
    for symbol, group in data_df.groupby('symbol'):
        daily_buys = group[group['transaction_type'] == 'Buy'].groupby('broadcast_date').filter(lambda x: len(x['insider_name'].unique()) >= 1)  # Relaxed to >=1
        signal_dates = daily_buys['broadcast_date'].unique()
        for date in signal_dates:
            tech_data = group[group['broadcast_date'] == date].iloc[0]
            all_signals.append({
                'date': date, 
                'symbol': symbol,
                'atr_10': tech_data.get('atr_10', 0.02),
                'adv_10': tech_data.get('adv_10', 10000000),
                'beta_60': tech_data.get('beta_60', 1.0),
                'volatility_60': tech_data.get('volatility_60', 0.2)
            })
    signal_df = pd.DataFrame(all_signals).sort_values('date').reset_index(drop=True)
    print(f"Identified {len(signal_df)} potential signal events across history.")

    sim_dates = pd.date_range(start=data_df['broadcast_date'].min(), end=sim_end_date)

    # Initialize equity curve
    equity_curve = pd.Series(index=sim_dates, dtype=float)

    for current_date in sim_dates:
        current_portfolio_value = cash
        for symbol, pos in holdings.items():
            stock_data = price_data_cache.get(symbol)
            if stock_data is not None and current_date in stock_data.index:
                current_portfolio_value += pos['shares'] * stock_data.loc[current_date, 'Close']
            else:
                current_portfolio_value += pos['shares'] * pos['entry_price']

        # Record equity for this date
        equity_curve[current_date] = current_portfolio_value

        for symbol, pos in list(holdings.items()):
            stock_data = price_data_cache.get(symbol)
            if stock_data is not None and current_date in stock_data.index:
                latest_price = stock_data.loc[current_date, 'Close']
                exit_reason = None
                if latest_price > pos['peak_price']:
                    pos['peak_price'] = latest_price
                    pos['date_of_peak_price'] = current_date
                
                trailing_stop = pos['peak_price'] - (1.5 * pos['atr_10'])  
                no_new_high_days = 25 if pos['volatility_60'] < 0.25 else 15  
                
                if latest_price >= pos['profit_target']:
                    exit_reason = "Profit Target"
                elif latest_price <= pos['stop_loss']:
                    exit_reason = "Stop Loss"
                elif (current_date - pos['entry_date']).days > 120:  
                    exit_reason = "Max Hold Exceeded"
                elif (current_date - pos['entry_date']).days > momentum_decay_min_hold_days and \
                     (current_date - pos['date_of_peak_price']).days > no_new_high_days:
                    exit_reason = "Momentum Decay"
                if exit_reason:
                    exit_value = (pos['shares'] * latest_price) * (1 - transaction_cost_pct)
                    cash += exit_value  
                    trade_log.append({
                        'Symbol': symbol,
                        'Entry Date': pos['entry_date'],
                        'Entry Price': pos['entry_price'],
                        'Exit Date': current_date,
                        'Exit Price': latest_price,
                        'Return (%)': ((latest_price / pos['entry_price']) - 1) * 100,
                        'Exit Reason': exit_reason,
                        'ATR': pos.get('atr_10', 0.02),
                        'Beta': pos.get('beta_60', 1.0),
                        'Volatility': pos.get('volatility_60', 0.2),
                        'ADV_Cr': pos.get('adv_10', 10000000) / 10000000
                    })
                    del holdings[symbol]

        todays_signals = signal_df[signal_df['date'] == current_date]
        if len(holdings) >= max_active_positions or todays_signals.empty:
            continue

        for _, signal in todays_signals.iterrows():
            symbol = signal['symbol']
            if symbol in holdings: continue
            historical_signals = signal_df[(signal_df['symbol'] == symbol) & (signal_df['date'] < current_date)]
            if len(historical_signals) < 1: continue  # Relaxed to >=1
            stock_data = price_data_cache.get(symbol)
            if stock_data is None: continue
            returns, volatilities = [], []
            for _, past_signal in historical_signals.tail(historical_events_lookback).iterrows():
                entry_date = past_signal['date'] + pd.Timedelta(days=1)
                exit_date = entry_date + pd.Timedelta(days=30)
                if entry_date in stock_data.index and exit_date in stock_data.index:
                    entry_price = stock_data.loc[entry_date, 'Open']
                    price_series = stock_data.loc[entry_date:exit_date, 'Close']
                    if entry_price > 0:
                        returns.append((price_series.iloc[-1] / entry_price) - 1)
                        volatilities.append(price_series.pct_change().std())
            if not returns or not volatilities: continue
            avg_return, avg_vol = np.mean(returns), np.mean(volatilities)
            if avg_return < -0.02:  # Less strict
                continue
            historical_win_rate = sum(1 for r in returns if r > 0) / len(returns) if returns else 0
            if historical_win_rate < 0.30:  # Lowered threshold
                print(f"  -> SKIPPED: {symbol} - Poor win rate: {historical_win_rate:.1%}")
                continue
            entry_date = current_date + pd.Timedelta(days=1)
            if entry_date not in stock_data.index: continue
            entry_price = stock_data.loc[entry_date, 'Open']
            if entry_price <= 0: continue
            profit_target_price = entry_price * (1 + max(avg_return, 0.15))  
            stop_loss_price = entry_price * (1 - (avg_vol * 4))  
            position_alloc = 0.05 if signal['beta_60'] > 1.5 else 0.08  
            position_size = current_portfolio_value * position_alloc
            shares_to_buy = position_size / entry_price
            trade_cost = shares_to_buy * entry_price * (1 + transaction_cost_pct)
            if cash >= trade_cost:
                cash -= trade_cost
                holdings[symbol] = {
                    'shares': shares_to_buy, 'entry_price': entry_price, 'entry_date': entry_date,
                    'profit_target': profit_target_price, 'stop_loss': stop_loss_price,
                    'peak_price': entry_price, 'date_of_peak_price': entry_date,
                    'atr_10': signal['atr_10'], 'beta_60': signal['beta_60'], 
                    'volatility_60': signal['volatility_60'], 'adv_10': signal['adv_10']
                }
                print(f"  -> TRADE: BUY {symbol} on {entry_date.date()} at {entry_price:.2f}")
                print(f"     -> Win Rate: {historical_win_rate:.1%}, Target: {profit_target_price:.2f}, Stop: {stop_loss_price:.2f}")
                if len(holdings) >= max_active_positions: break

 
    print("BACKTEST SIMULATION COMPLETE")
    end_time = time.time()
    print(f"Total execution time: {end_time - start_time:.2f} seconds.")

    if not trade_log:
        print("\nNo trades were executed. Final metrics cannot be calculated.")
    else:
        trade_log_df = pd.DataFrame(trade_log)
        print("\n--- Technical Indicators Summary ---")
        print(f"Average ATR: {trade_log_df['ATR'].mean():.3f}")
        print(f"Average Beta: {trade_log_df['Beta'].mean():.2f}")
        print(f"Average Volatility: {trade_log_df['Volatility'].mean():.3f}")
        print(f"Average ADV: {trade_log_df['ADV_Cr'].mean():.1f} Crores")

        print("\nPer-Stock Performance")

        def calculate_trade_drawdown(row):
            symbol, start, end = row['Symbol'], row['Entry Date'], row['Exit Date']
            if symbol not in price_data_cache or start not in price_data_cache[symbol].index or end not in price_data_cache[symbol].index:
                return 0
            prices = price_data_cache[symbol].loc[start:end, 'Close']
            if prices.empty: return 0
            peak = prices.cummax()
            drawdown = (prices / peak) - 1
            return drawdown.min() * 100

        trade_log_df['Max Drawdown (%)'] = trade_log_df.apply(calculate_trade_drawdown, axis=1)

        def profit_factor(s):
            wins = s[s > 0].sum()
            losses = np.abs(s[s < 0].sum())
            return wins / losses if losses > 0 else np.inf

        def trade_sharpe(s):
            if s.std() == 0 or len(s) < 2: return np.nan
            return s.mean() / s.std()

        stock_stats = trade_log_df.groupby('Symbol').agg(
            num_trades=('Symbol', 'size'),
            win_rate_pct=('Return (%)', lambda s: (s > 0).mean() * 100),
            avg_return_pct=('Return (%)', 'mean'),
            profit_factor=('Return (%)', profit_factor),
            avg_drawdown_pct=('Max Drawdown (%)', 'mean'),
            trade_sharpe=('Return (%)', trade_sharpe)
        ).sort_values('num_trades', ascending=False)

        stock_stats_display = stock_stats.copy()
        for col in ['win_rate_pct', 'avg_return_pct', 'avg_drawdown_pct']:
            stock_stats_display[col] = stock_stats_display[col].map('{:.2f}%'.format)
        for col in ['profit_factor', 'trade_sharpe']:
            stock_stats_display[col] = stock_stats_display[col].map('{:.2f}'.format)
        print(stock_stats_display.to_string())

        print("\n--- Detailed Trade Log ---")
        trade_log_df_display = trade_log_df.copy()
        trade_log_df_display['Entry Date'] = trade_log_df_display['Entry Date'].dt.date
        trade_log_df_display['Exit Date'] = trade_log_df_display['Exit Date'].dt.date
        trade_log_df_display['Entry Price'] = trade_log_df_display['Entry Price'].map('{:,.2f}'.format)
        trade_log_df_display['Exit Price'] = trade_log_df_display['Exit Price'].map('{:,.2f}'.format)
        trade_log_df_display['Return (%)'] = trade_log_df_display['Return (%)'].map('{:.2f}%'.format)
        trade_log_df_display['ATR'] = trade_log_df_display['ATR'].map('{:.3f}'.format)
        trade_log_df_display['Beta'] = trade_log_df_display['Beta'].map('{:.2f}'.format)
        display_columns = ['Symbol', 'Entry Date', 'Entry Price', 'Exit Date', 'Exit Price', 'Return (%)', 'Exit Reason', 'ATR', 'Beta']
        print(trade_log_df_display.to_string(columns=display_columns, index=False))

        # Add holding days to trade_log_df (for time adjustment)
        trade_log_df['holding_days'] = (trade_log_df['Exit Date'] - trade_log_df['Entry Date']).dt.days
        trade_log_df['holding_days'] = trade_log_df['holding_days'].clip(lower=1)  # Avoid division by zero
        
        total_return = (cash / initial_capital) - 1  # Use the final 'cash' value from backtest
        
        # CAGR (annualized over backtest span)
        if not trade_log_df.empty:
            first_entry = trade_log_df['Entry Date'].min()
            last_exit = trade_log_df['Exit Date'].max()
            years_span = (last_exit - first_entry).days / 365.25
            years_span = max(years_span, 1)  # Avoid issues with short spans
            cagr = (cash / initial_capital) ** (1 / years_span) - 1
        else:
            cagr = 0
        
        # Geometric Mean Return (compounded average per trade, for reference)
        geo_mean_return = (np.prod(1 + trade_log_df['Return (%)'] / 100) ** (1 / len(trade_log_df)) - 1) * 100 if not trade_log_df.empty else 0
        
        # Equity Curve-Based Metrics
        equity_curve = equity_curve.ffill().dropna()  # Fill any gaps
        daily_returns = equity_curve.pct_change().dropna()  # Daily % changes
        
        # Annualized Return (CAGR from equity curve, already calculated above, but confirm)
        mean_annual_return = (1 + daily_returns.mean()) ** 252 - 1
        
        # Downside Deviation for Sortino
        downside_returns = daily_returns[daily_returns < 0]
        downside_dev = downside_returns.std() * np.sqrt(252) if not downside_returns.empty else 0
        
        # Mean Annualized Excess Return
        mean_annual_excess = mean_annual_return - risk_free_rate
        
        # Sortino Ratio
        overall_sortino = mean_annual_excess / downside_dev if downside_dev > 0 else 0
        
        # Maximum Drawdown for Calmar
        rolling_max = equity_curve.cummax()
        drawdowns = (equity_curve / rolling_max) - 1
        max_drawdown = drawdowns.min() * -1  # Positive value
        overall_calmar = cagr / max_drawdown if max_drawdown > 0 else 0
        
        # Print Updated Metrics
        print("\nEnhanced Strategy Performance")
        print(f"Total Trades: {len(trade_log_df)}")
        print(f"Overall Compounded Return: {total_return * 100:.2f}%")
        print(f"Compounded Annual Growth Rate (CAGR): {cagr * 100:.2f}%")
        print(f"Geometric Mean Return per Trade: {geo_mean_return:.2f}%")
        print(f"Average Holding Days: {trade_log_df['holding_days'].mean():.1f}")
        print(f"Overall Annualized Sortino Ratio: {overall_sortino:.2f}")
        print(f"Overall Annualized Calmar Ratio: {overall_calmar:.2f}")


Loaded 'backtest_ready_data.csv' with 12297 rows.

Fetching all required price data...



*********************100%***********************]  913 of 913 completed

Successfully fetched and cached price data for 913 symbols.
Finding all potential historical signals...
Identified 4282 potential signal events across history.
  -> TRADE: BUY MAHSEAMLES on 2022-08-02 at 367.05
     -> Win Rate: 100.0%, Target: 422.11, Stop: 332.40
  -> TRADE: BUY ASTEC on 2022-08-04 at 1908.60
     -> Win Rate: 100.0%, Target: 2194.89, Stop: 1838.88
  -> TRADE: BUY TOKYOPLAST on 2022-08-04 at 89.90
     -> Win Rate: 100.0%, Target: 103.39, Stop: 76.64
  -> SKIPPED: AURIONPRO - Poor win rate: 0.0%
  -> TRADE: BUY GODREJAGRO on 2022-08-18 at 505.70
     -> Win Rate: 100.0%, Target: 581.56, Stop: 479.28
  -> TRADE: BUY NCLIND on 2022-08-18 at 175.00
     -> Win Rate: 100.0%, Target: 201.25, Stop: 163.67
  -> TRADE: BUY DBCORP on 2022-08-18 at 92.00
     -> Win Rate: 100.0%, Target: 110.29, Stop: 78.14
  -> TRADE: BUY CONFIPET on 2022-08-25 at 69.00
     -> Win Rate: 100.0%, Target: 83.73, Stop: 58.52
  -> TRADE: BUY VAIBHAVGBL on 2022-08-25 at 307.55
     -> Win Rate: 10